# Overlaps between R/U/GHF

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

In this notebook, we will explore the multi-configurational behaviour of UHF and GHF, by projecting the UHF and GHF ONVs onto the full ONV basis expressed with respect to the RHF orbitals.

## Projecting UHF

### H2

As an introductory example, let's start by examining H2. We'll set up an unrestricted spin-orbital basis, where the alpha- and beta-coefficient matrices are equal. Then, we expect the projection of the UHF ONV onto the ONV basis expressed with respect to the RHF spin-orbitals to yield a linear combination consisting of exactly one 1, and the rest 0.

In [2]:
molecule = gqcpy.Molecule.HChain(2, 1.0)
N = molecule.numberOfElectrons()

In [7]:
r_spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
K = r_spinor_basis.numberOfSpatialOrbitals()

S = r_spinor_basis.quantize(gqcpy.OverlapOperator())

In [5]:
sq_hamiltonian = r_spinor_basis.quantize(gqcpy.FQMolecularHamiltonian(molecule))  # 'sq' for 'second-quantized'

In [9]:
rhf_environment = gqcpy.RHFSCFEnvironment_d.WithCoreGuess(N, sq_hamiltonian, S)
rhf_solver = gqcpy.RHFSCFSolver_d.Plain()

rhf_objective = gqcpy.DiagonalRHFFockMatrixObjective_d(sq_hamiltonian)  # use the default threshold of 1.0e-08

In [11]:
rhf_parameters = gqcpy.RHF_d.optimize(rhf_objective, rhf_solver, rhf_environment).groundStateParameters()

After having done the RHF calculation, we are able to construct restricted and unrestricted spin-orbital bases.

In [12]:
r_spinor_basis.transform(rhf_parameters.expansion())

In [14]:
u_spinor_basis = gqcpy.USpinOrbitalBasis_d.FromRestricted(r_spinor_basis)

We are now in the position to calculate the expansion of the UHF determinant in the full spin-resolved ONV basis expressed with respect to the RHF spin-orbitals.

In [15]:
uhf_determinant = gqcpy.SpinResolvedONV.UHF(2, 1, 1)

Projecting this determinant is done by constructing a `LinearExpansion` from an ONV projection:

In [16]:
expansion = gqcpy.LinearExpansion_SpinResolved.FromONVProjection(uhf_determinant, r_spinor_basis, u_spinor_basis)

def print_function(coefficient, onv):
    print("{:7.4f}".format(coefficient), onv)
    
expansion.forEach(print_function)

 1.0000 01|01
-0.0000 01|10
-0.0000 10|01
 0.0000 10|10


### H4

As a more interesting example, we'll project the H4 UHF triplet state onto the basis of ONVs expressed with respect to the RHF spin-orbitals.

In [17]:
molecule = gqcpy.Molecule.HRingFromDistance(4, 1.0, 0)
N = molecule.numberOfElectrons()
N_P = N//2
N_alpha = N//2
N_beta = N//2

internuclear_repulsion_energy = gqcpy.NuclearRepulsionOperator(molecule.nuclearFramework()).value()

Using Xeno's code, we find the following coefficient matrices for the RHF solution:

In [18]:
C = [[-0.27745359, -0.8505133,   0.85051937,  2.02075317],
     [-0.27745362, -0.85051937, -0.8505133,  -2.02075317],
     [-0.27745359,  0.8505133,  -0.85051937,  2.02075317],
     [-0.27745362,  0.85051937,  0.8505133,  -2.02075317]]
C = np.array(C)

and the UHF solution:

In [19]:
C_alpha = [[-1.75646828e-01, -1.20606646e-06,  1.20281173e+00,  2.03213486e+00],
           [-3.78560533e-01, -1.20281173e+00, -1.20606647e-06, -2.00427438e+00],
           [-1.75646828e-01,  1.20606646e-06, -1.20281173e+00,  2.03213486e+00],
           [-3.78560533e-01,  1.20281173e+00,  1.20606646e-06, -2.00427438e+00]]
C_alpha = np.array(C_alpha)

C_beta = [[-3.78560533e-01,  1.20281173e+00,  1.21724557e-06,  2.00427438e+00],
          [-1.75646828e-01,  1.21724558e-06, -1.20281173e+00, -2.03213486e+00],
          [-3.78560533e-01, -1.20281173e+00, -1.21724558e-06,  2.00427438e+00],
          [-1.75646828e-01, -1.21724558e-06,  1.20281173e+00, -2.03213486e+00]]
C_beta = np.array(C_beta)

Let's now set up the corresponding restricted and unrestricted spin-orbital bases.

In [21]:
r_spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
r_spinor_basis.transform(gqcpy.RTransformation_d(C))

In [24]:
u_spinor_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
u_spinor_basis.transform(gqcpy.UTransformation_d(gqcpy.UTransformationComponent_d(C_alpha), gqcpy.UTransformationComponent_d(C_beta)))

We can now construct the UHF spin-resolved ONV:

In [25]:
uhf_determinant = gqcpy.SpinResolvedONV.UHF(4, 2, 2)

and project it onto the spin-resolved basis of ONVs expressed with respect to the orthonormal restricted spin-orbitals:

In [26]:
expansion = gqcpy.LinearExpansion_SpinResolved.FromONVProjection(uhf_determinant, r_spinor_basis, u_spinor_basis)

expansion.forEach(print_function)

-0.4987 0011|0011
 0.4987 0011|0101
 0.0000 0011|0110
 0.0000 0011|1001
-0.0251 0011|1010
 0.0251 0011|1100
-0.4987 0101|0011
 0.4987 0101|0101
 0.0000 0101|0110
 0.0000 0101|1001
-0.0251 0101|1010
 0.0251 0101|1100
-0.0000 0110|0011
 0.0000 0110|0101
 0.0000 0110|0110
 0.0000 0110|1001
-0.0000 0110|1010
 0.0000 0110|1100
 0.0000 1001|0011
-0.0000 1001|0101
-0.0000 1001|0110
-0.0000 1001|1001
 0.0000 1001|1010
-0.0000 1001|1100
 0.0251 1010|0011
-0.0251 1010|0101
-0.0000 1010|0110
-0.0000 1010|1001
 0.0013 1010|1010
-0.0013 1010|1100
 0.0251 1100|0011
-0.0251 1100|0101
-0.0000 1100|0110
-0.0000 1100|1001
 0.0013 1100|1010
-0.0013 1100|1100
